<a href="https://colab.research.google.com/github/sanilnadkarni/500-AI-Machine-learning-Deep-learning-Computer-vision-NLP-Projects-with-code/blob/main/System_Promptin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

!pip install --quiet openai>=1.40.0

import os, json, time
from openai import OpenAI


In [1]:
from google.colab import userdata
from openai import OpenAI

# Retrieve your secret key (make sure you have saved it in Colab as 'OPENAI_API_KEY')
api_key = userdata.get('OPENAI_API_KEY')

if api_key:
    print("✅ OpenAI API key loaded securely from Colab userdata.")
    client = OpenAI(api_key=api_key)
else:
    raise ValueError("❌ Could not find 'OPENAI_API_KEY' in Colab userdata. Please set it first.")


✅ OpenAI API key loaded securely from Colab userdata.


In [2]:
import json # Import the json module here

examples = [
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "My order is late"},
      {"role": "assistant", "content": "I’m sorry to hear that. Please share your order ID and I’ll check the status right away."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "Refund policy"},
      {"role": "assistant", "content": "You can return items within 30 days with proof of purchase. Would you like me to start a return?"}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "How do I change my shipping address?"},
      {"role": "assistant", "content": "Go to Account → Addresses → Edit, or tell me the new address and I’ll update it for your current order."}
    ]
  },
    {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "My product is broken"},
      {"role": "assistant", "content": "I'm sorry to hear that. Please provide details about the damage and your order ID so I can assist you with a replacement or return."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role to": "user", "content": "How can I track my order?"},
      {"role": "assistant", "content": "You can track your order using the tracking number provided in your shipping confirmation email. If you need help finding it, please provide your order ID."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "What are the shipping options?"},
      {"role": "assistant", "content": "We offer standard, expedited, and express shipping options. The cost and delivery time depend on your location and chosen method."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "Can I cancel my order?"},
      {"role": "assistant", "content": "Orders can be cancelled within 24 hours of placement, provided they haven't shipped yet. Please provide your order ID to check if cancellation is possible."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "Do you ship internationally?"},
      {"role": "assistant", "content": "Yes, we offer international shipping to many countries. Please enter your address during checkout to see if we ship to your location and to view shipping costs."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "How do I apply a discount code?"},
      {"role": "assistant", "content": "You can apply a discount code during the checkout process. There will be a field to enter the code before you finalize your purchase."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "What payment methods do you accept?"},
      {"role": "assistant", "content": "We accept major credit cards (Visa, Mastercard, American Express), PayPal, and other local payment methods depending on your region."}
    ]
  },
  {
    "messages": [
      {"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."},
      {"role": "user", "content": "My account is locked"},
      {"role": "assistant", "content": "I'm sorry to hear that your account is locked. Please provide your email address or username associated with the account and I can help you unlock it or reset your password."}
    ]
  }
]

with open("data.jsonl", "w") as f:
    for row in examples:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

!wc -l data.jsonl && head -n 2 data.jsonl

11 data.jsonl
{"messages": [{"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."}, {"role": "user", "content": "My order is late"}, {"role": "assistant", "content": "I’m sorry to hear that. Please share your order ID and I’ll check the status right away."}]}
{"messages": [{"role": "system", "content": "You are a helpful customer support agent for AcmeCo. Be concise and friendly."}, {"role": "user", "content": "Refund policy"}, {"role": "assistant", "content": "You can return items within 30 days with proof of purchase. Would you like me to start a return?"}]}


In [3]:
upload = client.files.create(
    file=open("data.jsonl", "rb"),
    purpose="fine-tune"
)

training_file_id = upload.id
print("Training file id:", training_file_id)


Training file id: file-3mMoKHx2oCANTCEefBFp29


In [4]:
job = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo"
)

job_id = job.id
print("Fine-tune job id:", job_id, "| status:", job.status)

Fine-tune job id: ftjob-I1yB2K2LxwZsgejnTl2mLI5L | status: validating_files


In [5]:
import time

def poll(job_id, interval=10):
    while True:
        j = client.fine_tuning.jobs.retrieve(job_id)
        print(f"[{time.strftime('%H:%M:%S')}] status={j.status}")
        if j.status in ("succeeded", "failed", "cancelled"):
            return j
        time.sleep(interval)

final = poll(job_id, interval=10)
print("Final status:", final.status)
print("Fine-tuned model:", final.fine_tuned_model)

[01:31:06] status=validating_files
[01:31:16] status=validating_files
[01:31:26] status=validating_files
[01:31:37] status=running
[01:31:47] status=failed
Final status: failed
Fine-tuned model: None


In [ ]:
events = client.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=50)

print("Fine-tuning job events:")
for event in events.data:
    print(f"- {event.created_at} - {event.message}")

Fine-tuning job events:
- 1756264074 - Training file has 3 example(s), but must have at least 10 examples
- 1756264025 - Validating training file: file-6JMj8MAQuMj5izuXJXyjYy
- 1756264025 - Created fine-tuning job: ftjob-7PHT0i0JPcNOwC1T1DTEIfQs


In [ ]:
# Rerun the cell to create the updated data.jsonl file
# cell_id: Kqmj_ABWe4DJ - This cell was already run by the user after the previous modification.
# I will proceed with the next steps.

# Upload the new data.jsonl file to OpenAI
upload = client.files.create(
    file=open("data.jsonl", "rb"),
    purpose="fine-tune"
)

training_file_id = upload.id
print("Training file id:", training_file_id)

Training file id: file-TPNmT5ZBwdywSkSM7TqtrQ


In [ ]:
# Create a new fine-tuning job with the updated training file
job = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-3.5-turbo"
)

job_id = job.id
print("Fine-tune job id:", job_id, "| status:", job.status)

Fine-tune job id: ftjob-FI3V75CMVI6eSbSlDVvyPYBU | status: validating_files


In [6]:
# Poll the job status until it completes
import time

def poll(job_id, interval=10):
    while True:
        j = client.fine_tuning.jobs.retrieve(job_id)
        print(f"[{time.strftime('%H:%M:%S')}] status={j.status}")
        if j.status in ("succeeded", "failed", "cancelled"):
            return j
        time.sleep(interval)

final = poll(job_id, interval=10)
print("Final status:", final.status)
print("Fine-tuned model:", final.fine_tuned_model)

[01:32:23] status=failed
Final status: failed
Fine-tuned model: None


In [ ]:
ft_model = final.fine_tuned_model
if not ft_model:
    raise RuntimeError("Fine-tune did not produce a model. Check logs/events above.")

resp = client.chat.completions.create(
    model=ft_model,
    messages=[
        {"role": "user", "content": "I want a refund for a damaged item."}
    ]
)

print("FT Response:\n", resp.choices[0].message.content)


RuntimeError: Fine-tune did not produce a model. Check logs/events above.